In [1]:
import os
from dotenv import load_dotenv

#load_dotenv()

#OPENAI_API_KEY = os.getenv("OPENAI_API_KEY") # (Optional), if OpenAI Model is used

MODEL = "mistral:latest" # Name of the model used by Ollama

COLLECTION_NAME = 'doc_qa_db' # Name of the Collection to be created

DIMENSION = 768 # Dimension of the embeddings

URI = 'http://localhost:19530' # Connection parameters for the Milvus Server

In [2]:
# Import LLM and Embeddings

from langchain_community.llms import Ollama
from langchain_community.embeddings import OllamaEmbeddings

model = Ollama(model=MODEL)
# print(model.invoke("Who is the fastest football player in the world?"))
embeddings = OllamaEmbeddings(model="mistral:latest")

In [3]:
# (Optional) When you use Open AI Model, you have to parse the output

from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

chain = model | parser
chain.invoke("Tell me a joke")

In [6]:
# (Example) Use a PDF and split it to later save it into the Vector Store and do Question Answering with it

from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("Optimierung Cloud-zentrierter Anwendungslandschaften durch Application Portfolio Management.pdf")
pages = loader.load_and_split()
pages

In [7]:
# Configure the prompt template that is used to ask the LLM

from langchain.prompts import PromptTemplate

template = """
Answer the question based on the context below. If you can't answer the question, reply "I don't know".

Context: {context}

Question: {question}
"""

prompt = PromptTemplate.from_template(template)
print(prompt.format(context="Here is some context", question="Here is a question"))

In [8]:
# Build a LangChain chain containing the prompt and the model

chain = prompt | model 
chain.input_schema.schema()

In [10]:
# Testing the chain

chain.invoke(
    {
        "context": "My name is Marvin",
        "question": "What is my girlfriends name?"
    }
)

In [13]:
# Use Milvus as Vectorstore

from langchain_community.vectorstores import Milvus

connection_args = {'uri': URI }

vectorstore = Milvus(
    embedding_function=embeddings,
    connection_args=connection_args,
    collection_name=COLLECTION_NAME,
    drop_old=True
).from_documents(
    pages,
    embedding=embeddings,
    collection_name=COLLECTION_NAME,
    connection_args=connection_args
)

In [16]:
# (Optional) When not using Milvus, this is a simple alternative for a basic vector store

from langchain_community.vectorstores import DocArrayInMemorySearch

vectorstore = DocArrayInMemorySearch.from_documents(
    pages,
    embedding=embeddings
)

In [22]:
# Do a similarity search for a query

query = "Software Lifecycle"
docs = vectorstore.similarity_search(query)

print(len(docs))

In [23]:
# Test by using the vectorstore as a retriever. The retriever gives back the relevant pages based on the query invoked

retriever = vectorstore.as_retriever()

retriever.invoke("LeanIX")

In [21]:
# Build a chain additionally context information from the Vector database

from operator import itemgetter
chain = (
    {
        "context": itemgetter("question") | retriever, 
        "question": itemgetter("question")
    }
    | prompt 
    | model
)

print(chain.invoke({"question": "Was ist LeanIX?"}))

In [28]:
# Alternative way to print the result so every character is printed single in a stream as in ChatGPT

for s in chain.stream({"question": "Was ist Cloud Computing?"}):
    print(s, end="", flush=True)